In [ ]:
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
import pandas as pd

# Importing data from Tecan excel file, needs the file location, concentraition of the injected solution, injection volume, starting volume and label (name) of the kinetic loop
def dataset(datafile, concentration, invol, vol0, label):

    # Open the file
    df = pd.read_excel("file")

    '''

    #Find the row with zero measurement
    label0_row_index = df.index[df.iloc[:, 0] == 'Temperature'].tolist()[0]

    # Generate the DataFrame using rows below the "Label 2" row until the first empty row
    start0_row_index = label0_row_index + 1

    # Create the DataFrame
    df0 = df.iloc[start0_row_index+1:start0_row_index+3]

    df0.columns = df0.iloc[0]
    df0 = df0[1:]

    df0 = df0.iloc[: , 1:]

    df0 = df0.dropna(axis='columns')

    '''

    # Finding the kinetic loop measurements
    # Find the row containing label in the first column
    label2_row_index = df.index[df.iloc[:, 0] == label].tolist()[0]

    # Generate the DataFrame using rows below the "Label 2" row until the first empty row
    start_row_index = label2_row_index + 1
    end_row_index = start_row_index
    while end_row_index < len(df) and not df.iloc[end_row_index, 0] is None:
        end_row_index += 1

    # Create the dataframe and append point zero
    filtered_df = df.iloc[start_row_index:end_row_index-2]

    filtered_df.columns = filtered_df.iloc[0]
    filtered_df = filtered_df[1:]

    '''
    df0.columns = filtered_df.columns.values[3:]
    df0['Cycle Nr.'] = 0
    full_df = df0.append(filtered_df)
    '''
    full_df = filtered_df

    full_df['Cycle Nr.'] = full_df['Cycle Nr.'].astype(int)
    full_df['Volume added [ul]'] = full_df['Cycle Nr.']*invol
    full_df['Conc. added [mM]'] = full_df['Volume added [ul]']*650/(full_df['Volume added [ul]']+100)

    full_df = full_df.apply(pd.to_numeric, errors='coerce')

    return full_df

def minslope(x, y):

    slopes = []
    bs = []

    #range for determining the max-slope point (larger range may be useful for noisy data)
    braket = 2

    for i in range(braket, len(x) - braket):
        slope, b = np.polyfit(x[i-braket:i+braket], y[i-braket:i+braket], 1)
        slopes.append(slope)
        bs.append(b)

    min_slope_index = np.argmin(slopes)  # Add 1 to account for starting from the second point

    return slopes[min_slope_index], bs[min_slope_index]

def baseline(x, y):

    baseline_indices = np.argsort(y)[:int(len(y) * 0.05)]
    baseline_x = x.values[baseline_indices]
    a, b = np.polyfit(baseline_x, y.values[baseline_indices], 1)
    return a, b

def find_intersection(a1, b1, a2, b2):

    x_intersect = (b2 - b1) / (a1 - a2)
    y_intersect = a1 * x_intersect + b1

    return x_intersect, y_intersect

def red_errors(e):

    is_error = e != 'all good'
    return ['background-color: red' if v else '' for v in is_error]

curdir = r'/content/Surfactant'
file = 'AsCatpep_CMP_0011.xlsx'
filedir = rf'{curdir}/'+file

df = dataset(filedir, 2, 2, 100, 'Label 2')

samples = df.columns[3:-2]

df_res = pd.DataFrame(columns = ['sample', 'CSC', 'comment', 'x_crit', 'y_crit'])

tot = len(samples)
cols = 4
rows = tot//cols

if tot % cols != 0:
    rows += 1

position = range(1,tot + 1)

fig = plt.figure(1, figsize=(20, rows*4), dpi=300)

for k, sample in enumerate(samples):

    if df[sample].max()-df[sample].min()<0.1:
        comment = 'too low absorbance at the start, probably no LLPS'
    else:
        comment = 'all good'

    A, B = minslope(df['Conc. added [mM]'], df[sample])
    a, b = baseline (df['Conc. added [mM]'], df[sample])

    X, Y = find_intersection(A, B, a, b)

    if comment == 'all good':
        CSC = X
    else:
        CSC = np.nan

    new_row = {'sample':sample, 'CSC':CSC, 'comment':comment, 'x_crit':X, 'y_crit':Y}

    df_res = pd.concat([df_res, pd.DataFrame([new_row])], ignore_index=True)

    ax = fig.add_subplot(rows, cols, position[k])

    ax.scatter(df['Conc. added [mM]'], df[sample])

    ax.plot(df['Conc. added [mM]'], B+A*df['Conc. added [mM]'])
    ax.plot(df['Conc. added [mM]'], b+a*df['Conc. added [mM]'])

    ax.set_ylim([0, 1])

    ax.set_title(sample)

    ax.set_xlabel('NaCl concentration added (mM)')
    ax.set_ylabel('absorbance at 600 nm')

    #if comment == 'all good':
    #    ax.text(X, 0.5, 'CSC='+str(X))

fig.tight_layout(pad=3.0)
fig.savefig(rf'{curdir}/fits.png')
fig.show()

print(df_res)

df_res.style.apply(red_errors, subset=['comment']).to_excel(rf'{curdir}/CSCs.xlsx')